In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../data/catastici.db")

In [4]:
import pandas as pd
questions = pd.read_csv("test_data.csv")
questions.head()

,question_id,level_len,level_nest,question,true_query,true_answer
0,0,0,0,What are the various kinds of properties ident...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."
1,0,0,0,Could you enumerate the different property cat...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."
2,0,0,0,"In the dataset, what property types can be found?","SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."
3,0,0,0,Specify all the property types that are encaps...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."
4,0,0,0,List all the types of properties in the dataset.,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('..."


In [5]:
import re
def get_evidence(question, db = db, fields = ["Owner_ID","Owner_First_Name","Owner_Family_Name","Property_Type","Rent_Income","Property_Location"]):
    keywords = re.findall(r'"(.*?)"', question)
    if len(keywords) == 0:
        return ""
    evidence = []
    for keyword in keywords:
        for field in fields:
            if len(db.run(f"""SELECT * FROM catastici WHERE "{field}" = "{keyword}" LIMIT 1;""")) > 0:
                evidence.append(f'"{field}" = "{keyword}"')
                break
    return "; ".join(evidence)

# example
print(questions['question'][360])
get_evidence(questions['question'][360])

What’s the minimum amount "zuane" "panizza" earns from the property at "traghetto di san silvestro"?


'"Owner_First_Name" = "zuane"; "Owner_Family_Name" = "panizza"; "Property_Location" = "traghetto di san silvestro"'

In [6]:
# Build evidence
evidences = []
for i, r in questions.iterrows():
    evidences.append(get_evidence(r['question']))
questions['evidence'] = evidences
questions.sample(5)

,question_id,level_len,level_nest,question,true_query,true_answer,evidence
203,40,0,0,"How wide is the span of rent profits on ""calle...","SELECT MIN(""Rent_Income""), MAX(""Rent_Income"") ...","[(8, 32)]","""Property_Location"" = ""calle de franchi"""
220,44,0,0,Who is the top earner from a single property i...,"SELECT ""Owner_First_Name"", ""Owner_Family_Name""...","[('antonio', 'danielli', 14934)]",
484,96,1,1,How many properties account for the lower 50% ...,"SELECT COUNT(""Property_Type"") \nFROM catastici...","[(7856,)]",
191,38,0,0,What does the average earning from rent look l...,"SELECT ""Property_Location"", AVG(""Rent_Income"")...",[('[appartamento] li formenti sopra la strada ...,
105,21,0,0,What is the total rent revenue generated from ...,"SELECT SUM(""Rent_Income"") \nFROM catastici \nW...","[(259,)]","""Property_Type"" = ""bottega da casarol"""


In [10]:
# questions['evidence'].unique()
questions.iloc[360]['question']+'\n'+questions.iloc[360]['evidence']+'\n'+questions.iloc[360]['true_query']

'What’s the minimum amount "zuane" "panizza" earns from the property at "traghetto di san silvestro"?\n"Owner_First_Name" = "zuane"; "Owner_Family_Name" = "panizza"; "Property_Location" = "traghetto di san silvestro"\nSELECT MIN("Rent_Income") \nFROM catastici \nWHERE "Owner_First_Name" = \'zuane\' \nAND "Owner_Family_Name" = \'panizza\' \nAND "Property_Location" = \'traghetto di san silvestro\';'

In [38]:
questions.to_csv("test_data_with_evidence.csv", index=False)